In [ ]:
from flask import Flask, render_template, Response
import cv2
import numpy as np
import tensorflow as tf

app = Flask(__name__, template_folder='.', static_folder = 'static')
bmi_model = tf.keras.models.load_model("model7.h5")
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Access the webcam
video_capture = cv2.VideoCapture(0)

def generate_frames():
    while True:
        # Capture frame-by-frame from the webcam
        ret, frame = video_capture.read()

        # Convert the frame to RGB
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Perform face detection
        faces = face_cascade.detectMultiScale(rgb_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        for (x, y, w, h) in faces:
            # Extract the face region
            face_image = rgb_frame[y:y+h, x:x+w]

            # Preprocess the face image
            face_image = cv2.resize(face_image, (224, 224))  # Adjust the size as per your model's input requirements
            face_image = np.expand_dims(face_image, axis=0)
            face_image = face_image / 255.0  # Normalize the image

            # Predict BMI
            bmi_prediction = bmi_model.predict(face_image)
            bmi_value = bmi_prediction[0]
            bmi_text = f'BMI: {float(bmi_value):.2f}'
            cv2.putText(frame, f'{bmi_text}', (x, y-30), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 2)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # Encode the frame as JPEG
        ret, buffer = cv2.imencode('.jpg', frame)
        frame = buffer.tobytes()

        # Yield the frame as an HTTP response
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/video_feed')
def video_feed():
    return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    app.run(ssl_context='adhoc', host='0.0.0.0', port=8000)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on https://127.0.0.1:8000
 * Running on https://192.168.1.247:8000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:192.168.1.247 - - [01/Mar/2024 14:05:01] "GET / HTTP/1.1" 200 -
INFO:werkzeug:192.168.1.247 - - [01/Mar/2024 14:05:01] "GET /static/BMI_Background.jpg HTTP/1.1" 404 -
INFO:werkzeug:192.168.1.247 - - [01/Mar/2024 14:05:01] "GET /video_feed HTTP/1.1" 200 -


1/1 [==============================] - 0s 44ms/step


ERROR:werkzeug:Error on request:
Traceback (most recent call last):
  File "/Users/sirivanth/anaconda3/lib/python3.10/site-packages/werkzeug/serving.py", line 335, in run_wsgi
    execute(self.server.app)
  File "/Users/sirivanth/anaconda3/lib/python3.10/site-packages/werkzeug/serving.py", line 325, in execute
    write(data)
  File "/Users/sirivanth/anaconda3/lib/python3.10/site-packages/werkzeug/serving.py", line 300, in write
    self.wfile.write(data)
  File "/Users/sirivanth/anaconda3/lib/python3.10/socketserver.py", line 826, in write
    self._sock.sendall(b)
  File "/Users/sirivanth/anaconda3/lib/python3.10/ssl.py", line 1237, in sendall
    v = self.send(byte_view[count:])
  File "/Users/sirivanth/anaconda3/lib/python3.10/ssl.py", line 1206, in send
    return self._sslobj.write(data)
ssl.SSLEOFError: EOF occurred in violation of protocol (_ssl.c:2396)
